In [1]:
def load_my_modules(prj_name=None, lst=None):

    import sys, os

    if prj_name==None:  prj_name = 'project_KnowledgeBase'       
    if lst==None:       lst = [
            '\\my_methods',
            '\\personal_expenses\\modules'
            # '\\qqq,',        
    ]

    # Получить текущую папку
    cwd = os.getcwd()
    prj_len = len(prj_name)
    i1 = cwd.find(prj_name)
    prj_dir = cwd[0:i1+prj_len]
    # d:\IDE_prj\project_KnowledgeBase

    # '\' -> '\\'
    if not prj_dir.find('\\\\'):
        prj_dir = prj_dir.replace('\\', '\\\\')
    
    # Проверить наличие модулей в sys.path_hook
    # Если отсутствют - добавить
    for module_dir in lst:
        module_dir = prj_dir + module_dir
        if not module_dir in sys.path:
            sys.path.insert(0, module_dir)
    return sys.path

load_my_modules()

['d:\\IDE_prj\\project_KnowledgeBase\\personal_expenses\\modules',
 'd:\\IDE_prj\\project_KnowledgeBase\\my_methods',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39\\DLLs',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39\\lib',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39',
 '',
 'C:\\Users\\Александр\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'C:\\Users\\Александр\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32',
 'C:\\Users\\Александр\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32\\lib',
 'C:\\Users\\Александр\\AppData\\Roaming\\Python\\Python39\\site-packages\\Pythonwin',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages']

In [19]:
import xml.etree.ElementTree as ET
import pandas as pd
import datetime as dt
import my_methods as mm

file_name = 'data/test_rnkb_from_xlsx.xml'
xlm = ET.parse(file_name)
tag_root = xlm.getroot()
for tag in tag_root:
    print(tag)

level =   '{urn:schemas-microsoft-com:office:spreadsheet}Worksheet'
level += '/{urn:schemas-microsoft-com:office:spreadsheet}Table'
level += '/{urn:schemas-microsoft-com:office:spreadsheet}Row'


<Element '{urn:schemas-microsoft-com:office:office}DocumentProperties' at 0x000001B75CB50590>
<Element '{urn:schemas-microsoft-com:office:office}OfficeDocumentSettings' at 0x000001B75CB50D60>
<Element '{urn:schemas-microsoft-com:office:excel}ExcelWorkbook' at 0x000001B75CB50EF0>
<Element '{urn:schemas-microsoft-com:office:spreadsheet}Styles' at 0x000001B75CB4B220>
<Element '{urn:schemas-microsoft-com:office:spreadsheet}Names' at 0x000001B75CB4F680>
<Element '{urn:schemas-microsoft-com:office:spreadsheet}Worksheet' at 0x000001B75CB4F630>


In [ ]:
# Print rows
for i_row, tag_row in enumerate(tag_root.findall(level)):
    print('row ', i_row, '-----------------', tag_row)
    for text in tag_row.itertext():
        print(text)

In [ ]:
# Print rows
for i_row, tag_row in enumerate(tag_root.findall(level)):
    # if i_row >=11:
        print('row ', i_row, '-----------------', tag_row)
        for i_cel, tag in enumerate(tag_row.findall('{urn:schemas-microsoft-com:office:spreadsheet}Cell')):            
            text = ''
            for t in tag.itertext():
                text = t
            print(text)
            # print(tag)
                    

In [36]:
import datetime
s='01.04.2022 08:39'
datetime.datetime.strptime(s, '%d.%m.%Y %H:%M')

datetime.datetime(2022, 4, 1, 8, 39)

In [46]:
import datetime

def load_payments_from_XML_RNKB(file_name):
    # возвращает словарь результатов:
    # df: DataFrame
    # Параметры выписки (d1, d2, sum1, sum2, приходы, расходы, метка карты)
    # res['success'] = True - флаг успешного чтения формата файла

    res = {}
    res["error_text"] = "" # значение поумолчанию, т.е. нет ошибок
    res['success'] = False
    res['prop'] = [None, None,0,0]  # res['prop'] [Остаток1=None, Остаток2=None, Расходы, Приходы]


    # tag_root
    xlm = ET.parse(file_name)
    tag_root = xlm.getroot()

    level =   '{urn:schemas-microsoft-com:office:spreadsheet}Worksheet'
    level += '/{urn:schemas-microsoft-com:office:spreadsheet}Table'
    level += '/{urn:schemas-microsoft-com:office:spreadsheet}Row'

    rows = []#{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[]}

    # считать данные в список
    for tag_row in tag_root.findall(level):        
        row = []
        for i_cel, tag in enumerate(tag_row.findall('{urn:schemas-microsoft-com:office:spreadsheet}Cell')):            
            text = ''
            for t in tag.itertext():
                text = t
            row.append(text)
        rows.append(row)
    
    if len(rows) < 11:
        res['error_text'] = 'При попытке чтения XML файла возникла ошибка формата файла. Недостаточная длина строк (код 101)'
        res['success'] = False
        return res

    # Заголовки
    # Найти метку
    mark_text = 'Карта'
    if rows[1][0] != mark_text:
        res['error_text'] = 'Ошибка формата файла. Не найдена метка (код 102)'
        res['success'] = False
        return res

    # Найти метку карты (в той же строке)
    res['card_mark'] = rows[1][1]

    # 'd1', 'd2', 'period'
    res['d1'] = datetime.datetime.strptime(rows[3][1], '%d.%m.%Y')
    res['d2'] = datetime.datetime.strptime(rows[4][1], '%d.%m.%Y') + datetime.timedelta(days=1)
    res['period'] = f'за период с xx.xx.20xx по xx.xx.20xx'

    # res['prop'] [Остаток1=None, Остаток2=None, Расходы, Приходы]
    # "Поступления"
    res['prop'][3] = mm.text_to_float(rows[5][1])
    # "Расходы"
    res['prop'][2] = mm.text_to_float(rows[6][1])

# row  11 ----------------- <Element '{urn:schemas-microsoft-com:office:spreadsheet}Row' at 0x000001B75BE28C20>
# [0] 01.04.2022 08:39
# [1] 01.04.2022 00:00
# [2] 
# [3] -50 000.00
# [4] 0.00
# [5] -50 000.00
# [6] RUR
# [7] Другое
# [8]
# row  12 ----------------- <Element '{urn:schemas-microsoft-com:office:spreadsheet}Row' at 0x000001B75BE2F270>
# [0] 01.04.2022 14:37
# [1] 01.04.2022 00:00
# [2] Выдача наличных RUS SEVASTOPOL ATM-8316
# [3] -49 000.00
# [4] 0.00
# [5] -49 000.00
# [6] RUR
# [7] Снятие наличных
# [8]

    rows2 = []
    n1 = len(rows)
    for ind, r1 in enumerate(rows[11:n1]):
        row = []
        
        # date = DateTimeField()
        val = dt.datetime.strptime(r1[0], '%d.%m.%Y %H:%M')
        row.append(val)

        # date_account = DateTimeField(null=True)    
        val = dt.datetime.strptime(r1[1], '%d.%m.%Y %H:%M')
        row.append(val)

        # category = CharField(null=True) 
        val = r1[7]
        row.append(val)

        # name = CharField(null=True)
        val = r1[2]
        row.append(val)

        # amount = FloatField()
        st = r1[3]
        val = mm.text_to_float(st)
        row.append(val)

        # balance
        val = None
        row.append(val)

        # Сохранить строку
        rows2.append(row)

    # res['df']
    # Создать DataFrame из списка rows
    columns=["date", "date_account", "category", "name", "amount", "balance"]
    df = pd.DataFrame(columns=columns, data=rows2).sort_values(by='date')
    res['df'] = df
    res['mess'] = f"Записать/обновить банковскую выписку? \n\n\
            Карта: {res['card_mark']}\n\
            {res['period']}\n\
            Приходы: {res['prop'][3]:_.2f}  \n\
            Расходы: {res['prop'][2]:_.2f}".replace('_', ' ')
    res['success'] = True
    # df.to_csv('test_out2.csv', sep=';', decimal=',', encoding='cp1251')

    return res
    

load_payments_from_XML_RNKB(file_name)    

{'error_text': '',
 'success': True,
 'prop': [None, None, 461698.81, 19416.61],
 'card_mark': 'Карта Дебетовая MASS N *2043',
 'd1': datetime.datetime(2022, 4, 1, 0, 0),
 'd2': datetime.datetime(2022, 4, 27, 0, 0),
 'period': 'за период с xx.xx.20xx по xx.xx.20xx',
 'df':                   date date_account                  category  \
 0  2022-04-01 08:39:00   2022-04-01                    Другое   
 1  2022-04-01 14:37:00   2022-04-01           Снятие наличных   
 2  2022-04-02 10:24:00   2022-04-04       Коммунальные услуги   
 3  2022-04-02 13:15:00   2022-04-04              Супермаркеты   
 4  2022-04-02 13:15:00   2022-04-04  Одежда, обувь, украшения   
 ..                 ...          ...                       ...   
 59 2022-04-24 10:09:00   2022-04-25              Супермаркеты   
 60 2022-04-24 10:09:00   2022-04-25  Одежда, обувь, украшения   
 61 2022-04-24 19:24:00   2022-04-25              Супермаркеты   
 62 2022-04-26 06:02:00   2022-04-26                    Другое   
 

In [22]:
import datetime

def load_payments_from_XML_RNKB(file_name):
    # возвращает словарь результатов:
    # df: DataFrame
    # Параметры выписки (d1, d2, sum1, sum2, приходы, расходы, метка карты)
    # res['success'] = True - флаг успешного чтения формата файла

    res = {}
    res["error_text"] = "" # значение поумолчанию, т.е. нет ошибок
    res['success'] = False
    res['prop'] = [None, None,0,0]  # res['prop'] [Остаток1=None, Остаток2=None, Расходы, Приходы]


    # tag_root
    xlm = ET.parse(file_name)
    tag_root = xlm.getroot()

    level =   '{urn:schemas-microsoft-com:office:spreadsheet}Worksheet'
    level += '/{urn:schemas-microsoft-com:office:spreadsheet}Table'
    level += '/{urn:schemas-microsoft-com:office:spreadsheet}Row'

    rows = []#{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[]}

    flag_mark = False  # Найти текст 'Карта'
    flag_card = False  # Найти метку карты
    for i_row, tag_row in enumerate(tag_root.findall(level)):
        
        # flag1_mark  Найти метку в строке с заданным номером
        mark_text = 'Карта'
        if i_row == 1:
            for i, text in enumerate(tag_row.itertext()):
                if i == 2:                    
                    if text == mark_text:
                        flag_mark = True
                    else: 
                        break

                # Найти метку карты (в той же строке)
                elif i == 5:
                    if len(text) < 4:
                        break
                    # res['card_mark']
                    res['card_mark'] = text
                    flag_card = True        # допуск на дальнейший парсинг
                    break
            if not flag_mark or not flag_card:
                res.error_text = 'При попытке чтения XML файла в строке 4 не найдена метка "Карта" или название карты (код 001)'
                res['success'] = False
                return res

        # 'd1'
        elif i_row == 3:
            for i, text in enumerate(tag_row.itertext()):
                if i == 5:
                    res['d1'] = datetime.datetime.strptime(text, '%d.%m.%Y')
                break
        # 'd2', 'period'
        elif i_row == 4:
            for i, text in enumerate(tag_row.itertext()):
                if i == 5:
                    res['d1'] = datetime.datetime.strptime(text, '%d.%m.%Y') + datetime.timedelta(days=1)
                    res['period'] = f'за период с xx.xx.20xx по xx.xx.20xx'
                break
        
        # res['prop'] [Остаток1=None, Остаток2=None, Расходы, Приходы]
        # "Поступления"
        elif i_row == 5:
            for i, text in enumerate(tag_row.itertext()):
                if i == 5:
                    val = mm.text_to_float(text)
                    res['prop'][3] = val
                break
        # "Расходы"
        elif i_row == 6:
            for i, text in enumerate(tag_row.itertext()):
                if i == 5:
                    val = mm.text_to_float(text)
                    res['prop'][2] = val
                break


        elif i_row >= 19:        
            # print('row ', i_row, '-----------------', tag_row)

            row = []
            for i_cel, tag in enumerate(tag_row.findall('{urn:schemas-microsoft-com:office:spreadsheet}Cell')):            
                if i_cel > 7: break
                text = ''
                for t in tag.itertext():
                    text = t
                row.append(text)
            # row.insert(0, len(row))
            if len(row) == 8:
                rows.append(row)


    # Обработать rows в массив записей rows_ar
    len1 = len(rows)
    len2 = len1//2
    if len1%2 != 0:
        res.error_text = f'При попытке чтения XML файла считано не четное количество строк в таблице: {len1}'
        return res
    rows2 = []

    #   Date	    Time  ...   Name	...	         Amount	....	Balance	Amount_num	Balance_num
    # 0	2022-03-31	15:44		Перевод с карты		5 000,00		337.94	5000.00	337.94
    # 1	2022-03-31	202880		SBOL перевод 2202****6497 В. СЕРГЕЙ СЕРГЕЕВИЧ					NaN	NaN
    for i2 in range(0, len2):
        i1 = 2*i2
        r1 = rows[i1]
        r2 = rows[i1+1]    
        row = []
        
        # date = DateTimeField()
        val = dt.datetime.strptime(r1[0] + ' ' + r1[1], '%Y-%m-%d %H:%M')
        row.append(val)

        # date_account = DateTimeField(null=True)    
        val = dt.datetime.strptime(r2[0], '%Y-%m-%d')
        row.append(val)

        # category = CharField(null=True) 
        val = r1[3]
        row.append(val)

        # name = CharField(null=True)
        val = r2[3]
        row.append(val)

        # amount = FloatField()
        st = r1[5]
        val = mm.text_to_float(st) if st[0] == '+' else -mm.text_to_float(st)
        row.append(val)

        # balance
        val = mm.text_to_float(r1[7])
        row.append(val)

        # Сохранить в массив 
        rows2.append(row)

    # res['df']
    # Создать DataFrame из списка rows
    columns=["date", "date_account", "category", "name", "amount", "balance"]
    df = pd.DataFrame(columns=columns, data=rows2).sort_values(by='date')
    res['df'] = df
    res['mess'] = f"Записать/обновить банковскую выписку? \n\n\
            Карта: {res['card_mark']}\n\
            {res['period']}\n\
            Приходы: {res['prop'][3]:_.2f}  \n\
            Расходы: {res['prop'][2]:_.2f}  \n\
            Остаток на начло периода: { res['prop'][0]:_.2f}\n\
            Остаток на конец периода: { res['prop'][1]:_.2f}".replace('_', ' ')
    res['success'] = True
    # df.to_csv('test_out2.csv', sep=';', decimal=',', encoding='cp1251')

    return res
    
load_payments_from_XML_RNKB(file_name)


KeyboardInterrupt

